In [55]:
%pwd

'd:\\mlops\\etoe1ds'

In [56]:
from box import box
from box import Box,ConfigBox
data=Box({"name":"Devesh",
      "rno.":"27",
    "room":2,
    "address":{
        "block":"Bward, Jail Road",
        "FlatNo.":"301,C-wing",
        "BuildingName":"LullaComplex"
    }
    
      })

data.address

Box({'block': 'Bward, Jail Road', 'FlatNo.': '301,C-wing', 'BuildingName': 'LullaComplex'})

In [57]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir:Path
    source_url:str
    local_data_file:Path
    unzip_dir:Path

In [58]:
from src.datascience.constants import *
from src.datascience.utils.common import  read_yaml ,create_directories

In [59]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [60]:
import os
os.chdir("/mlops/etoe1ds")

In [61]:
%pwd

'd:\\mlops\\etoe1ds'

In [62]:
import os
from src.datascience import logger
import urllib.request as request
import zipfile

In [63]:
#component for data ingestion
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers =request.urlretrieve(
                url = self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename } downloaded")
        else:
            logger.info(f"FIles is present")
    def extract_zip_file(self):
        unzip_path= self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r')as zip_ref:
            zip_ref.extractall(unzip_path)

In [64]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2025-01-29 16:03:34,999: INFO: common: yaml file: config\config.yaml loading successfull]
[2025-01-29 16:03:35,002: INFO: common: yaml file: params.yaml loading successfull]
[2025-01-29 16:03:35,006: INFO: common: yaml file: schema.yaml loading successfull]
[2025-01-29 16:03:35,008: INFO: common: Cretaed directory at:artifacts]
[2025-01-29 16:03:35,009: INFO: common: Cretaed directory at:artifacts/data_ingestion]
[2025-01-29 16:03:35,011: INFO: 750986838: FIles is present]


In [65]:
##data validation

In [66]:
%pwd

'd:\\mlops\\etoe1ds'

In [67]:
import pandas as pd

data=pd.read_csv("artifacts/data_ingestion/winequality-red.csv")

In [68]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [69]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [70]:
data.shape

(1599, 12)

In [71]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataValidationConfig:
    root_dir:Path
    STATUS_FILE:str
    unzip_data_dir:Path
    all_schema:dict

In [72]:
from src.datascience.utils.common import read_yaml,create_directories
from src.datascience.constants import *


In [73]:
class ValidationConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root,self.config.data_validation.root_dir])
    
    def get_data_validation_config(self)->DataValidationConfig:
        config=self.config.data_validation
        schema=self.schema.columns
        data_validation_config= DataValidationConfig(
               root_dir=config.root_dir,
               STATUS_FILE=config.STATUS_FILE,
               unzip_data_dir=config.unzip_data_dir,
               all_schema=schema
        )
        return data_validation_config

    

In [74]:
import os
from src.datascience import logger

In [75]:
class DataValidator:
    def __init__(self,config:DataValidationConfig):
        self.config=config
    
    def validate_all_columns(self)-> bool:
        try:
            validation_status=None
            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols= list(data.columns)
            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status= False
                    with open(self.config.STATUS_FILE,'w')as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status= True
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f"Validation Status:{validation_status}")
                   
            
            return validation_status
        except Exception as e:
            raise e

            

In [76]:
from src.datascience import logger

In [77]:
try:
    logger.info("Data Validation has been started")
    obj=ValidationConfigurationManager()
    data_validation_config=obj.get_data_validation_config()
    data_validator=DataValidator(config=data_validation_config)
    output=data_validator.validate_all_columns()
    logger.info("Data Validation has ended")
    print(output)
except Exception as e:
    logger.error(f"Data Validation error: {e}")
    raise e

[2025-01-29 16:03:35,315: INFO: 2505702399: Data Validation has been started]
[2025-01-29 16:03:35,324: INFO: common: yaml file: config\config.yaml loading successfull]
[2025-01-29 16:03:35,327: INFO: common: yaml file: params.yaml loading successfull]
[2025-01-29 16:03:35,331: INFO: common: yaml file: schema.yaml loading successfull]
[2025-01-29 16:03:35,333: INFO: common: Cretaed directory at:artifacts]
[2025-01-29 16:03:35,334: INFO: common: Cretaed directory at:artifacts/data_validation]
[2025-01-29 16:03:35,343: INFO: 2505702399: Data Validation has ended]


True


In [78]:
##Transformation

In [79]:
from src.datascience.utils.common import read_yaml,create_directories

In [80]:
from dataclasses import dataclass

@dataclass
class DataTransformationConfig:
    root_dir:Path
    data_path:Path

In [81]:
from src.datascience.constants import *

In [82]:
class TransformationConfigurationManager:
    def __init__(self,
                config_path=CONFIG_FILE_PATH,
                params_path=PARAMS_FILE_PATH,
                schema_path=SCHEMA_FILE_PATH
                ):
        self.config=read_yaml(config_path)
        self.params=read_yaml(params_path)
        self.schema=read_yaml(schema_path)
        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self)->DataTransformationConfig:
        config=self.config.data_transformation
        create_directories([config.root_dir])
        data_transformation_config=DataTransformationConfig(
           root_dir=config.root_dir,
           data_path=config.data_path
        )
        return data_transformation_config
        

In [83]:
import os
from src.datascience import logger
from sklearn.model_selection import train_test_split
import pandas as pd

    

In [84]:
class DataTransformation:
    def __init__(self,config:DataTransformationConfig):
        self.config=config

    def splitting_data(self):
        data=pd.read_csv(config.data_path)
        train,test=train_test_split(data)
        train.to_csv(os.path.join(self.config.root_dir,"train.csv"),index=False)
        test.to_csv(os.path.join(self.config.root_dir,"test.csv"),index=False)
        logger.info("Splitted the data into train and test")
        logger.info(f"Train Size:{train.shape}")
        logger.info(f"Test Size:{test.shape}")

        

In [85]:
try:
    logger.info(f"Data Transformation has started")
    obj=TransformationConfigurationManager()
    config=obj.get_data_transformation_config()
    DataTransformation(config).splitting_data()
    logger.info(f"Data Transformation ended ,data saved to artifacts.")
except Exception as e:
    raise e

[2025-01-29 16:03:35,550: INFO: 435105784: Data Transformation has started]
[2025-01-29 16:03:35,554: INFO: common: yaml file: config\config.yaml loading successfull]
[2025-01-29 16:03:35,557: INFO: common: yaml file: params.yaml loading successfull]
[2025-01-29 16:03:35,562: INFO: common: yaml file: schema.yaml loading successfull]
[2025-01-29 16:03:35,565: INFO: common: Cretaed directory at:artifacts]
[2025-01-29 16:03:35,567: INFO: common: Cretaed directory at:artifacts/data_transformations]
[2025-01-29 16:03:35,615: INFO: 1575699340: Splitted the data into train and test]
[2025-01-29 16:03:35,616: INFO: 1575699340: Train Size:(1199, 12)]
[2025-01-29 16:03:35,618: INFO: 1575699340: Test Size:(400, 12)]
[2025-01-29 16:03:35,620: INFO: 435105784: Data Transformation ended ,data saved to artifacts.]


In [87]:
##Model training
# import os
# os.chdir("mlops/etoe1ds")
# %pwd

In [88]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.model_selection import GridSearchCV

In [94]:
from dataclasses import dataclass

#  root_dir: artifacts/model_trainer
#   train_data_path: artifacts/data_transformation/train.csv
#   test_data_path: artifacts/data_transformation/test.csv
#   model_name: model.joblib

@dataclass
class DataTraningConfig:
    root_dir:Path
    train_data_path:Path
    test_data_path:Path
    model_name:Path
    alpha:float
    l1_ratio:float
    target_column: str


In [99]:
from src.datascience.utils.common import *
class TrainerConfigurationManager:
    def __init__(self,
        config_path =CONFIG_FILE_PATH,
        params_path= PARAMS_FILE_PATH,
        schema_path= SCHEMA_FILE_PATH    
                 ):
        self.config=read_yaml(config_path)
        self.params=read_yaml(params_path)
        self.schema=read_yaml(schema_path)
    def get_data_training_config(self)->DataTraningConfig:
        
        config=self.config.model_trainer
        create_directories([config.root_dir])
        params=self.params.ElasticNet
        schema =self.schema.target_value
        output=DataTraningConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.l1_ratio,
            l1_ratio=params.l1_ratio,
            target_column=schema.name
        )
        return output

In [103]:
import pandas as pd
import os
from src.datascience.utils.common import save_model
from src.datascience import logger
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
class Trainer:
    def __init__(self,config=DataTraningConfig):
        self.config=config

    def train_model(self):
        train_data= pd.read_csv(self.config.train_data_path)
        test_data= pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column],axis=1)
        test_x = test_data.drop([self.config.target_column],axis=1)
        train_y= train_data[self.config.target_column]
        test_y= test_data[self.config.target_column]
        lr = ElasticNet(alpha=self.config.alpha,l1_ratio=self.config.l1_ratio,random_state=42)
        lr.fit(train_x,train_y)
        joblib.dump(lr,os.path.join(self.config.root_dir,self.config.model_name))
        



In [104]:
try:
    obj= TrainerConfigurationManager()
    config=obj.get_data_training_config()
    Trainer(config).train_model()
except Exception as e:
    print(e)


[2025-01-29 16:08:34,970: INFO: common: yaml file: config\config.yaml loading successfull]
[2025-01-29 16:08:34,973: INFO: common: yaml file: params.yaml loading successfull]
[2025-01-29 16:08:34,976: INFO: common: yaml file: schema.yaml loading successfull]
[2025-01-29 16:08:34,978: INFO: common: Cretaed directory at:artifacts/model_trainer]


In [ ]:
##model evaluation

Accessing as SharmajiKabetaDevesh

Initialized MLflow to track repo "SharmajiKabetaDevesh/Endtoend_Data_science-project_1"

Repository SharmajiKabetaDevesh/Endtoend_Data_science-project_1 initialized!

In [14]:
from dotenv import load_dotenv
load_dotenv()
import os
os.chdir("etoe1ds")
%pwd

'd:\\Mlops\\etoe1ds'

In [15]:
access_key_id = os.getenv("access_key_id")
secret_access_key = os.getenv("secret_access_key")
MLFLOW_TRACKING_URI= os.getenv("MLFLOW_TRACKING_URI")
USER_NAME = os.getenv("USER_NAME")


In [24]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mflow_uri: str

In [25]:
from src.datascience.utils.common import save_json,read_yaml,create_directories

In [29]:
from src.datascience.constants import *
class ModelEvaluationConfigManager:
    def __init__(self,
         config_path=CONFIG_FILE_PATH,
         params_path=PARAMS_FILE_PATH,
         schema_path=SCHEMA_FILE_PATH
                 ):
        self.config=read_yaml(config_path)
        self.params=read_yaml(params_path)
        self.schema=read_yaml(schema_path)

    def get_evaluation_config_data(self):
        config=self.config.model_evalaution
        params=self.params.ElasticNet
        schema = self.schema.target_value
        create_directories([config.root_dir])
        output=ModelEvaluationConfig(
        root_dir=config.root_dir,
    test_data_path = config.test_data_path,
    model_path=config.model_path,
    all_params ={"alpha":params.alpha,
                 "l1_ratio":params.l1_ratio},
    metric_file_name=config.metric_file_name,
    target_column= schema.name,
    mflow_uri=os.getenv("MLFLOW_TRACKING_URI"),
            )
        return output



In [44]:
import pandas as pd
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from urllib.parse import urlparse

import joblib
import numpy as np

class EvaluateModel:
    def __init__(self,config=ModelEvaluationConfig):
        self.config=config
    
    def eval_metrics(self,actual,pred):
        rmse= np.sqrt(mean_squared_error(actual,pred))
        mae= mean_absolute_error(actual,pred)
        r2= r2_score(actual,pred)
        return rmse,mae,r2
    
    def log_data_to_mlflow(self):
        test_data=pd.read_csv(self.config.test_data_path)
        model=joblib.load(self.config.model_path)

        test_x= test_data.drop([self.config.target_column],axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mflow_uri)
        tracking_uri=urlparse(mlflow.get_tracking_uri()).scheme
        with mlflow.start_run():
            predicted_value= model.predict(test_x)
            (rmse,mae,r2)=self.eval_metrics(test_y,predicted_value)

            scores= {"rmse":rmse,"mae":mae, "r2":r2}
            save_json(path=Path(self.config.metric_file_name),data=scores)
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse",rmse)
            mlflow.log_metric("mae",mae)
            mlflow.log_metric("r2",r2)
            if tracking_uri!="file":
                mlflow.sklearn.log_model(model,"model",registered_model_name="ElasticNet")
            else:
                mlflow.sklearn.log_model(model,"model")    








In [45]:
try:
    obj= ModelEvaluationConfigManager()
    config=obj.get_evaluation_config_data()
    EvaluateModel(config).log_data_to_mlflow()
    

except Exception as e:
    print(e)

[2025-01-29 17:12:51,906: INFO: common: yaml file: config\config.yaml loading successfull]
[2025-01-29 17:12:51,908: INFO: common: yaml file: params.yaml loading successfull]
[2025-01-29 17:12:51,912: INFO: common: yaml file: schema.yaml loading successfull]
[2025-01-29 17:12:51,914: INFO: common: Cretaed directory at:artifacts/model_evaluation]
[2025-01-29 17:12:52,363: INFO: common: json file saved at artifacts\model_evaluation\metric.json]
2025/01/29 17:13:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticNet'.
2025/01/29 17:13:15 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticNet, version 1
Created version '1' of model 'ElasticNet'.


🏃 View run respected-moth-77 at: https://dagshub.com/SharmajiKabetaDevesh/Endtoend_Data_science-project_1.mlflow/#/experiments/0/runs/447b5218853448169a43e0f9037f0035
🧪 View experiment at: https://dagshub.com/SharmajiKabetaDevesh/Endtoend_Data_science-project_1.mlflow/#/experiments/0


In [46]:
import joblib
import numpy as np
import pandas as pd
from pathlib import Path

class PredictionPipeline:
    def __init__(self):
        self.model =joblib.load(Path('artifacts\model_trainer\model.joblib'))
    
    def predict(self,data):
        prediction=self.model.predict(data)
        return prediction 

In [48]:
obj=PredictionPipeline()
data = [7.4,0.7,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4]
data=np.array(data).reshape(1,11)
predict=obj.predict(data)

d:\Mlops\ETOE1DS\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but ElasticNet was fitted with feature names
  warnings.warn(


In [49]:
print(predict)

[5.21050808]
